In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import deeplake
from time import time
import pandas as pd


In [2]:
def LoadData(DATA, batch_size):

    if DATA == "CIFAR10":
        root = '../Data/CIFAR10'
        num_classes = 10

        cifar10_train_transform = transforms.Compose([
            transforms.RandomCrop(32),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
        cifar10_test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

        train = torchvision.datasets.CIFAR10(root=root, train=True, transform=cifar10_train_transform)
        test = torchvision.datasets.CIFAR10(root=root, train=False, transform=cifar10_test_transform)

        trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=2)
        testloader = torch.utils.data.DataLoader(test, batch_size=batch_size,shuffle=False, num_workers=2)
        print(DATA + " successfully loaded.")

    elif DATA == "CIFAR100":
        root = '../Data/CIFAR100'
        num_classes = 100

        cifar100_train_transform = transforms.Compose([
            transforms.RandomResizedCrop(32, scale=(0.8, 1.2)), # Simulate scale variations
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(degrees=15), # Consider for object orientations
            transforms.RandomGrayscale(p=0.2), # Optional for color augmentation
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Standard CIFAR-100 normalization
        ])
        cifar100_test_transform = transforms.Compose([
            # transforms.Resize(32),
            # transforms.CenterCrop(32),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        train = torchvision.datasets.CIFAR100(root=root, train=True, transform=cifar100_train_transform)
        test = torchvision.datasets.CIFAR100(root=root, train=False, transform=cifar100_test_transform)

        trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=2)
        testloader = torch.utils.data.DataLoader(test, batch_size=batch_size,shuffle=False, num_workers=2)
        print(DATA + " successfully loaded.")

    elif DATA == "IMAGENET":
        root = '../Data/IMAGENET'
        num_classes = 200

        tiny_imagenet_train_transform = transforms.Compose([
            transforms.RandomResizedCrop(64, scale=(0.8, 1.2), interpolation=transforms.InterpolationMode.BILINEAR),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ])
        tiny_imagenet_test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ])

        train = deeplake.load("hub://activeloop/tiny-imagenet-train")
        test = deeplake.load("hub://activeloop/tiny-imagenet-test")

        train.pytorch(transform = tiny_imagenet_train_transform, batch_size = batch_size, shuffle = True)
        test.pytorch(transform = tiny_imagenet_test_transform, batch_size = batch_size, shuffle = False)
        print(DATA + " successfully loaded.")

    elif DATA == "MNIST":
        root='../Data/MNIST'
        num_classes = 10

        mnist_train_transform = transforms.Compose([
            transforms.RandomRotation(10),  # Improve robustness
            transforms.RandomHorizontalFlip(),  # Augment data
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])


        mnist_test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])

        train = torchvision.datasets.MNIST(root=root, train=True, transform=mnist_train_transform, download=True)
        test = torchvision.datasets.MNIST(root=root, train=False, transform=mnist_test_transform ,download=True)

        trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
        testloader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
        print(DATA + " successfully loaded.")

    elif DATA == "FASION_MNIST":
        root='../Data/FASION_MNIST'
        num_classes = 10

        fashion_mnist_train_transform = transforms.Compose([
            transforms.RandomRotation(15),  # Improve robustness
            transforms.RandomHorizontalFlip(),  # Augment data
            transforms.RandomResizedCrop(28),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
        fashion_mnist_test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

        train = torchvision.datasets.FashionMNIST(root=root, train=True, transform=fashion_mnist_train_transform, download=True)
        test = torchvision.datasets.FashionMNIST(root=root, train=False, transform=fashion_mnist_test_transform ,download=True)

        trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
        testloader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
        print(DATA + " successfully loaded.")

    elif DATA == "PascalVOC":
        root = '../Data/PascalVOC'
        num_classes = 20

        voc_train_transform = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=(0.8, 1.2)),
            transforms.RandomHorizontalFlip(),  # Randomly flip for horizontal invariance
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])

        voc_test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])

        train = torchvision.datasets.VOCDetection(root=root, year='2012', image_set='trainval', transform=voc_train_transform, download = True)
        test = torchvision.datasets.VOCDetection(root=root, year='2012', image_set='trainval', transform=voc_test_transform, download = True)

        trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
        testloader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
        print(DATA + " successfully loaded.")

    elif DATA == "Flowers102":
        root = '../Data/Flowers102'
        num_classes = 102

        oxford102_train_transform = transforms.Compose([
            transforms.Resize(256),  # Adjust size based on your model
            transforms.RandomResizedCrop(224, scale=(0.8, 1.2)),  # Simulate scale variations
            transforms.RandomHorizontalFlip(),  # Increase data diversity
            transforms.RandomRotation(15),  # Consider for specific flower orientations (optional)
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),  # Introduce color variations
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))  # ImageNet stats, adjust if needed
        ])


        oxford102_test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])


        train = torchvision.datasets.Flowers102(root=root, transform=oxford102_train_transform, download=True)
        test = torchvision.datasets.Flowers102(root=root, transform=oxford102_test_transform, download = True)

        trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
        testloader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
        print(DATA + " successfully loaded.")

    return trainloader, testloader, num_classes

In [3]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Bottleneck, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)
        
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3 = nn.BatchNorm2d(out_channels*self.expansion)
        
        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()
        
    def forward(self, x):
        identity = x.clone()
        x = self.relu(self.batch_norm1(self.conv1(x)))
        
        x = self.relu(self.batch_norm2(self.conv2(x)))
        
        x = self.conv3(x)
        x = self.batch_norm3(x)
        
        #downsample if needed
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        #add identity
        x+=identity
        x=self.relu(x)
        
        return x

In [6]:
class ResNet(nn.Module):
    def __init__(self, ResBlock, layer_list, num_classes, num_channels=3):
        super(ResNet, self).__init__()
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size = 3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(ResBlock, layer_list[0], planes=64)
        self.layer2 = self._make_layer(ResBlock, layer_list[1], planes=128, stride=2)
        self.layer3 = self._make_layer(ResBlock, layer_list[2], planes=256, stride=2)
        self.layer4 = self._make_layer(ResBlock, layer_list[3], planes=512, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*ResBlock.expansion, num_classes)
        
    def forward(self, x):
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x
        
    def _make_layer(self, ResBlock, blocks, planes, stride=1):
        ii_downsample = None
        layers = []
        
        if stride != 1 or self.in_channels != planes*ResBlock.expansion:
            ii_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, planes*ResBlock.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes*ResBlock.expansion)
            )
            
        layers.append(ResBlock(self.in_channels, planes, i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*ResBlock.expansion
        
        for i in range(blocks-1):
            layers.append(ResBlock(self.in_channels, planes))
            
        return nn.Sequential(*layers)

In [7]:
def ResNet50(num_classes, channels=3):
    return ResNet(Bottleneck, [3,4,6,3], num_classes, channels)

In [8]:
def train(model, dataloader, criterion, optimizer):
    total = 0.
    correct = 0.
    running_loss = 0.
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        _, predicted = outputs.max(1)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        running_loss += loss.item()

    train_accuracy = 100 * correct / total
    train_loss = running_loss / total

    return train_loss, train_accuracy

In [9]:
def test(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total += labels.size(0)

            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            running_loss += loss.item()
    test_accuracy = 100 * correct / total
    test_loss = running_loss / total

    return test_loss, test_accuracy

In [10]:
# Data_List = ["CIFAR10", "CIFAR100", "IMAGENET", "MNIST", "FASION_MNIST", "PascalVOC", "Flowers102"]
Data_List = ["FASION_MNIST"]

for DATA in Data_List:
    trainloader, testloader, num_classes = LoadData(DATA, 128)

    model = ResNet50(num_classes, 1).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)

    last_epoch = 0
    num_epochs = 100
    history = []

    for epoch in range(last_epoch,num_epochs):
        t0 = time()
        model.train(True)
        running_loss = 0.0
        correct = 0
        total = 0
        train_loss, train_accuracy = train(model, trainloader, criterion, optimizer)
        t1 = time()
        test_loss, test_accuracy = test(model, testloader, criterion)
        t2 = time()

        data = {
            "Epoch": epoch + 1,
            "Train Loss": train_loss,
            "Train Accuracy": train_accuracy,
            "Train Time": t1 - t0,
            "Test Loss": test_loss,
            "Test Accuracy": test_accuracy,
            "Test Time": t2 - t1,
        }
        print(data)
        history.append(data)

    pd.DataFrame(history).to_json(f'./History/{DATA}-ResNet50-new.json')

FASION_MNIST successfully loaded.
{'Epoch': 1, 'Train Loss': 0.028952527197202045, 'Train Accuracy': 27.748333333333335, 'Train Time': 23.109874486923218, 'Test Loss': 0.010996375453472138, 'Test Accuracy': 50.1, 'Test Time': 1.3141021728515625}
{'Epoch': 2, 'Train Loss': 0.011845728472868602, 'Train Accuracy': 45.66, 'Train Time': 22.030632734298706, 'Test Loss': 0.007964900481700897, 'Test Accuracy': 63.37, 'Test Time': 1.2753195762634277}
{'Epoch': 3, 'Train Loss': 0.009815096282958985, 'Train Accuracy': 54.015, 'Train Time': 22.05880308151245, 'Test Loss': 0.006524019545316696, 'Test Accuracy': 68.84, 'Test Time': 1.2862281799316406}
{'Epoch': 4, 'Train Loss': 0.009028515060742696, 'Train Accuracy': 57.358333333333334, 'Train Time': 22.07662844657898, 'Test Loss': 0.005680880844593048, 'Test Accuracy': 71.69, 'Test Time': 1.307210922241211}
{'Epoch': 5, 'Train Loss': 0.008493875975410143, 'Train Accuracy': 59.626666666666665, 'Train Time': 22.333816528320312, 'Test Loss': 0.0057534